In [21]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd  

In [22]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [23]:
def cspan(IDTEXT): 
    span = 6
    if IDTEXT[0] == '0':
        return '6', '' 
    IDTEXT=IDTEXT.split('.')
    if len(IDTEXT) == 1:
        return '3','' 
    if len(IDTEXT) == 2:
        return '2',f'<td colspan="1">&nbsp</td>'   
    if len(IDTEXT) > 2:
        return '1',f'<td colspan="2">&nbsp</td>' 
    return str(span), ele
cspan('1.2.3')   

('1', '<td colspan="2">&nbsp</td>')

In [24]:
query = """ \
  	SELECT DISTINCT  
	Q_TypeCode AS FTPE,  IDTEXT,  SortPos, 
	PK_Question [{0}], 
	CASE WHEN ISNULL(IDTEXT, '') <> '' THEN
		REPLACE(REPLACE(IDTEXT, '.','_'), ' ','_') 
	ELSE
		'SDESC_' + CONVERT(NVARCHAR(9), SortPos)
	END AS [{1}], 
	CONVERT(NVARCHAR(5), PK_PickListType) + '' AS [{2}],
    '3' AS [{3}],
	CONVERT(NVARCHAR(9), PK_Question) + ' ' +  LEFT( REPLACE(REPLACE(QTEXT, ', Describe',''), 'If ','') , 15) AS [{4}],
	' r_idt_' + REPLACE(REPLACE(IDTEXT, '.','_'), ' ','_') +  ' ct_' +  Q_TypeCode  AS [{5}]
	FROM  vwQuestions WHERE PK_FORM LIKE '2022-A-HVA' AND SectionNum=1  ORDER BY SortPos ASC
"""  
df = get_df(query) 
df.fillna("",inplace=True)
df.replace(to_replace=[None], value=np.nan, inplace=True) 
df = df.astype(str)
df

,FTPE,IDTEXT,SortPos,{0},{1},{2},{3},{4},{5}
0,SDESC,,0,28400,SDESC_0,,3,28400 Mission Critica,r_idt__ ct_SDESC
1,MULTICHECKBOX,1B 1,1,28401,1B_1,,3,28401 What mission-es,r_idt_1B_1 ct_MULTICHECKBOX
2,TXT,1B 1.1,2,28402,1B_1_1,,3,"28402 Other, please d",r_idt_1B_1_1 ct_TXT
3,MULTICHECKBOX,1B 2,3,28403,1B_2,,3,28403 Does your HVA I,r_idt_1B_2 ct_MULTICHECKBOX
4,TXT,1B 2.1,4,28404,1B_2_1,,3,"28404 Other, please d",r_idt_1B_2_1 ct_TXT
...,...,...,...,...,...,...,...,...,...
83,TXT,1B 36.1,109,28539,1B_36_1,,3,"28539 Other, please d",r_idt_1B_36_1 ct_TXT
84,PICK,1B 37,110,28540,1B_37,,3,28540 Is remote acces,r_idt_1B_37 ct_PICK
85,TXT,1B 38,111,28541,1B_38,,3,28541 How is remote a,r_idt_1B_38 ct_TXT
86,MULTICHECKBOX,1B 39,112,28542,1B_39,,3,28542 Is there a dedi,r_idt_1B_39 ct_MULTICHECKBOX


In [25]:
df.to_csv(r'C:\Users\Tim\Documents\1802\HVA\form.csv', index=False)  
df[0:4]

,FTPE,IDTEXT,SortPos,{0},{1},{2},{3},{4},{5}
0,SDESC,,0,28400,SDESC_0,,3,28400 Mission Critica,r_idt__ ct_SDESC
1,MULTICHECKBOX,1B 1,1,28401,1B_1,,3,28401 What mission-es,r_idt_1B_1 ct_MULTICHECKBOX
2,TXT,1B 1.1,2,28402,1B_1_1,,3,"28402 Other, please d",r_idt_1B_1_1 ct_TXT
3,MULTICHECKBOX,1B 2,3,28403,1B_2,,3,28403 Does your HVA I,r_idt_1B_2 ct_MULTICHECKBOX


In [26]:
df = pd.read_csv(r'C:\Users\Tim\Documents\1802\HVA\form.csv', dtype=str)  
df.fillna("",inplace=True)
df = df.astype(str) 
df[0:32]

,FTPE,IDTEXT,SortPos,{0},{1},{2},{3},{4},{5}
0,SDESC,,0,28400,SDESC_0,,3,28400 Mission Critica,r_idt__ ct_SDESC
1,MULTICHECKBOX,1B 1,1,28401,1B_1,,3,28401 What mission-es,r_idt_1B_1 ct_MULTICHECKBOX
2,TXT,1B 1.1,2,28402,1B_1_1,,3,"28402 Other, please d",r_idt_1B_1_1 ct_TXT
3,MULTICHECKBOX,1B 2,3,28403,1B_2,,3,28403 Does your HVA I,r_idt_1B_2 ct_MULTICHECKBOX
4,TXT,1B 2.1,4,28404,1B_2_1,,3,"28404 Other, please d",r_idt_1B_2_1 ct_TXT
5,MULTICHECKBOX,1B 3,5,28405,1B_3,,3,28405 What types of s,r_idt_1B_3 ct_MULTICHECKBOX
6,TXT,1B 3.1,6,28406,1B_3_1,,3,"28406 Other, please d",r_idt_1B_3_1 ct_TXT
7,PICK,1B 4,7,28407,1B_4,,3,28407 your ICS enviro,r_idt_1B_4 ct_PICK
8,TXT,1B 4.1,8,28408,1B_4_1,,3,"28408 Other, please d",r_idt_1B_4_1 ct_TXT
9,PICK,1B 5,9,28409,1B_5,,3,28409 Are there any n,r_idt_1B_5 ct_PICK


In [27]:

st=''
for i,r in df.iterrows():
    tmp = f"C:\_som\_src\{r['FTPE']}.aspx"
    r0 = r['{0}']
    with open(tmp, 'r') as f:
        tmp = f.read()
    csp = cspan(r["IDTEXT"])
    tmp = re.sub('colspan="\d"','colspan="{3}"',tmp)
    tmp = tmp.replace("<!--{cols}-->", str(csp[1])) 
    tmp = tmp.replace("{3}", str(csp[0])) 
    tmp = tmp.replace("{0}", str(r["{0}"])) 
    tmp = tmp.replace("{1}", str(r["{1}"])) 
    tmp = tmp.replace("{2}", str(r["{2}"]))  
    tmp = tmp.replace("{4}", str(r["{0}"]) + ' ' + str(r["{4}"]))  
    tmp = tmp.replace("{5}", str(r["{5}"]))  
    st=st+tmp
with open(r'C:\Users\Tim\Documents\1802\HVA\page.aspx', 'w') as f:
    f.write(st) 

In [28]:
f1A=''
with open(fr'C:\_som\_src\1A.aspx', mode="r", encoding="utf-8") as f:
    f1A = f.read()  
f1A = f1A.replace("<!--BODY-->", st)
with open(r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\HVA\2022\2022_A_HVA_1B.aspx', 'w') as f:
    f.write(f1A) 
print( f1A )

<%@ Page Title="" Language="vb" AutoEventWireup="false" MasterPageFile="~/Form.Master" CodeBehind="2022_A_HVA_1B.aspx.vb" Inherits="CyberScope._2022_A_HVA_1B" %>

<%@ Register Src="../../CustomControls/CBlabel.ascx" TagName="CBlabel" TagPrefix="uc1" %>
<%@ Register Src="../../CustomControls/CBoptionalLinks.ascx" TagName="CBoptionalLinks" TagPrefix="uc6" %>
<%@ Register Src="../../CustomControls/CBButtPanel2.ascx" TagName="CBButtPanel" TagPrefix="uc8" %>
<%@ Register Src="../../CustomControls/CBtext2.ascx" TagName="CBtext2" TagPrefix="uc2" %>
<%@ Register Src="../../CustomControls/CBPickList2.ascx" TagName="CBPickList2" TagPrefix="uc5" %>
<%@ Register Src="../../CustomControls/CbPickListMultiSelect.ascx" TagName="CbPickListMultiSelect" TagPrefix="uc9" %>
<%@ Register Src="../../CustomControls/CBYesNo.ascx" TagName="CBYesNo" TagPrefix="uc3" %>
<%@ Register Src="../../CustomControls/CBArtifact.ascx" TagPrefix="uc1" TagName="CBArtifact" %>
<asp:Content ID="Content1" ContentPlaceHolderID="C